
# Kraken + ALTO OCR — Colab (Per‑Manuscript Projects, Best‑Model Saver)

This notebook keeps **everything per manuscript** inside a dedicated Drive folder and trains **recognition** models on ALTO data.

**What you get**
- One-time **Project ID** → creates `MyDrive/kraken_projects/<PROJECT_ID>/`
- Upload ALTO **ZIP** → auto-extracts to `data/`
- ALTO-first pairing → writes `train.txt` and `val.txt` to `lists/`
- **Controls** for: attempt number, learning rate, base model (previous/manual/auto-other)
- Training uses **`-t`/`-e`** (Kraken 3) and after training **copies the best checkpoint** to
  `models/rec/attempt_XX.mlmodel`
- CPU speed tweaks, lean Drive usage (small pip cache)


## 1) Connect Google Drive
This connects your Google Drive so the notebook can read/write your project files.
You’ll be asked to authorize once (click the link, choose your account, allow access).

Outcome: Drive mounted at /content/drive.

Tip: If you see a warning about “already mounted,” it’s fine.

In [1]:

from google.colab import drive  # type: ignore
drive.mount('/content/drive')
print("✅ Drive mounted at /content/drive")


Mounted at /content/drive
✅ Drive mounted at /content/drive



## 2) Project Settings (edit only `PROJECT_ID`)

- `PROJECT_ID`: your manuscript ID (e.g., `0093`).  
This cell creates the project folder tree on Drive:
```
MyDrive/kraken_projects/<PROJECT_ID>/
├── data/          # upload & extracted dataset (ALTO XML + images)
├── lists/         # train.txt, val.txt
└── models/
    └── rec/       # attempt_01.mlmodel, attempt_02.mlmodel, ...
```

What to edit: only the PROJECT_ID (your manuscript number, e.g., 0093).
Outcome: Project folders created; paths printed.

In [16]:

#@title 🔧 Project Settings
from pathlib import Path
import os

PROJECT_ID = "0093"  #@param {type:"string"}

ROOT_IN_DRIVE = "/content/drive/MyDrive"
PROJECTS_ROOT = f"{ROOT_IN_DRIVE}/kraken_projects"
PROJECT_DIR   = f"{PROJECTS_ROOT}/{PROJECT_ID}"
DATA_DIR      = f"{PROJECT_DIR}/data"
LISTS_DIR     = f"{PROJECT_DIR}/lists"
REC_MODELS    = f"{PROJECT_DIR}/models/rec"
PIP_CACHE_DIR = f"{ROOT_IN_DRIVE}/.pip-cache"   # small cache only

# Create the full project tree
for p in [PROJECTS_ROOT, PROJECT_DIR, DATA_DIR, LISTS_DIR, REC_MODELS, PIP_CACHE_DIR]:
    Path(p).mkdir(parents=True, exist_ok=True)

TRAIN_LIST = f"{LISTS_DIR}/train.txt"
VAL_LIST   = f"{LISTS_DIR}/val.txt"

# CPU basics
CORES = os.cpu_count() or 2
CPU_THREADS = max(2, CORES - 1)
DEVICE = "cpu"   # set "cuda" if you enable a T4 GPU in Colab

print("✅ Project folders ready")
print("PROJECT_DIR:", PROJECT_DIR)
print("DATA_DIR:", DATA_DIR)
print("LISTS_DIR:", LISTS_DIR)
print("REC_MODELS:", REC_MODELS)
print("CPU_THREADS:", CPU_THREADS, "| CORES:", CORES)


✅ Project folders ready
PROJECT_DIR: /content/drive/MyDrive/kraken_projects/0093
DATA_DIR: /content/drive/MyDrive/kraken_projects/0093/data
LISTS_DIR: /content/drive/MyDrive/kraken_projects/0093/lists
REC_MODELS: /content/drive/MyDrive/kraken_projects/0093/models/rec
CPU_THREADS: 2 | CORES: 2


## 3) CPU Speed Boost (threads & math libs)

Sets thread/env variables so image and math libraries don’t oversubscribe CPU cores.
Nothing to edit.
Outcome: Printed thread settings (this can shave seconds on CPU).

In [17]:

import os
os.environ["OMP_NUM_THREADS"] = str(CPU_THREADS)
os.environ["MKL_NUM_THREADS"] = str(CPU_THREADS)
os.environ["OPENBLAS_NUM_THREADS"] = str(CPU_THREADS)
os.environ["NUMEXPR_NUM_THREADS"] = str(CPU_THREADS)
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "0"
os.environ["KMP_AFFINITY"] = "granularity=fine,compact,1,0"

for k in ["OMP_NUM_THREADS","MKL_NUM_THREADS","OPENBLAS_NUM_THREADS","NUMEXPR_NUM_THREADS","KMP_BLOCKTIME","KMP_AFFINITY"]:
    print(k, "=", os.environ.get(k))


OMP_NUM_THREADS = 2
MKL_NUM_THREADS = 2
OPENBLAS_NUM_THREADS = 2
NUMEXPR_NUM_THREADS = 2
KMP_BLOCKTIME = 1
KMP_AFFINITY = granularity=fine,compact,1,0


## 4) Install Kraken (lean) + Pillow‑SIMD (faster image I/O)
Installs Kraken OCR and a faster Pillow build for image I/O.
The cell only installs if Kraken isn’t present.
Nothing to edit.
Outcome: “Kraken available/installed” and “Pillow-SIMD installed”.

In [18]:

# Install Kraken (lean) + Pillow-SIMD (faster image IO)

import os, subprocess, sys

# Lean pip settings
os.environ["PIP_CACHE_DIR"] = PIP_CACHE_DIR
os.environ["PIP_DISABLE_PIP_VERSION_CHECK"] = "1"
os.environ["PIP_NO_INPUT"] = "1"

def is_importable(pkg: str) -> bool:
    try:
        __import__(pkg)
        return True
    except Exception:
        return False

# Install kraken only if missing
if is_importable("kraken"):
    import kraken
    print(f"✅ Kraken available (version: {getattr(kraken, '__version__', 'unknown')})")
else:
    print("⏳ Installing Kraken (lean) ...")
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "--upgrade", "pip"], check=True)
    subprocess.run([
        sys.executable, "-m", "pip", "-q", "install",
        "--prefer-binary",
        "--upgrade-strategy", "only-if-needed",
        "kraken[cairo]"
    ], check=True)
    import kraken, importlib
    importlib.reload(kraken)
    print(f"✅ Installed Kraken (version: {getattr(kraken, '__version__', 'unknown')})")

# Switch to Pillow-SIMD for faster image decoding/resizing
print("⏳ Switching to Pillow-SIMD for faster image ops...")
# Don't fail the whole cell if Pillow isn't present yet
subprocess.run([sys.executable, "-m", "pip", "-q", "uninstall", "-y", "pillow"], check=False)
subprocess.run([
    sys.executable, "-m", "pip", "-q", "install",
    "--prefer-binary",
    "--upgrade-strategy", "only-if-needed",
    "pillow-simd"
], check=True)
print("✅ Pillow-SIMD installed")


✅ Kraken available (version: unknown)
⏳ Switching to Pillow-SIMD for faster image ops...
✅ Pillow-SIMD installed


## 5) Upload your ALTO dataset (ZIP → Drive project folder)
Upload your ZIP containing images and ALTO XML files.
The ZIP is extracted into your project’s data/ folder on Drive.

What to do: Click the upload button and pick your ZIP.
Outcome: File listing of a few extracted files.

Your ZIP should contain paired page images (e.g., .tif/.jpg) and ALTO XMLs. The XMLs must include <fileName> elements referring to the page image.

In [19]:

from google.colab import files  # type: ignore
import zipfile, os

print("📦 Please select your ZIP (ALTO XML + images)...")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("❌ No file uploaded.")

zip_name = next(iter(uploaded.keys()))
zip_path = f"/content/{zip_name}"

# Extract into the Drive project data folder
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(DATA_DIR)

print(f"✅ Extracted into: {DATA_DIR}")
!find "$DATA_DIR" -maxdepth 2 -type f | head -n 20


📦 Please select your ZIP (ALTO XML + images)...


Saving export_doc5946_0093_alto_202510261625 2-20251119T220539Z-1-001.zip to export_doc5946_0093_alto_202510261625 2-20251119T220539Z-1-001 (2).zip
✅ Extracted into: /content/drive/MyDrive/kraken_projects/0093/data
/content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0020_mirrored.xml
/content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0025_mirrored.jpg
/content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0023_mirrored.xml
/content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0022_mirrored.xml
/content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0026_mirrored.jpg
/content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0023_mirrored.jpg
/content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0024_mirrored.xml
/content/drive/MyDrive/kraken_projec

## 6) Build train/val lists from ALTO (in project folder)
Scans data/ for ALTO XMLs and writes:
	•	lists/train.txt (90%)
	•	lists/val.txt (10%)

Important: For -f alto, lists must contain only XML file paths, one per line—no image paths.
Outcome: Counts printed + a short sample of each list.

If it says “Not enough ALTO XMLs” or shows missing files, check your ZIP structure.

In [20]:
# Build train/val lists for ALTO: ONE XML PATH PER LINE (no image paths)

from pathlib import Path
import xml.etree.ElementTree as ET
import random, os

random.seed(42)  # reproducible split

DATA_DIR = DATA_DIR  # already defined
LISTS_DIR = LISTS_DIR
TRAIN_LIST = TRAIN_LIST
VAL_LIST = VAL_LIST

def is_alto_xml(p: Path) -> bool:
    if p.suffix.lower() != ".xml" or not p.is_file():
        return False
    try:
        # quick scan first
        with open(p, "r", encoding="utf-8", errors="ignore") as fh:
            head = fh.read(4096)
            if "<alto" not in head:
                return False
        # light parse to be safe
        ET.parse(p)
        return True
    except Exception:
        return False

# Collect all ALTO XMLs under DATA_DIR
xmls = [p for p in Path(DATA_DIR).rglob("*.xml") if is_alto_xml(p)]
xmls = sorted({x.resolve() for x in xmls})  # dedup + sort

n = len(xmls)
print(f"Found {n} ALTO XML files.")

if n < 2:
    raise SystemExit(f"❌ Not enough ALTO XMLs in {DATA_DIR}. Found {n}.")

# 90/10 split, shuffled
random.shuffle(xmls)
cut = max(1, int(n * 0.9))
train_xmls, val_xmls = xmls[:cut], xmls[cut:]

Path(LISTS_DIR).mkdir(parents=True, exist_ok=True)
with open(TRAIN_LIST, "w", encoding="utf-8") as f:
    for x in train_xmls:
        f.write(str(x) + "\n")
with open(VAL_LIST, "w", encoding="utf-8") as f:
    for x in val_xmls:
        f.write(str(x) + "\n")

print(f"✅ Wrote lists:")
print(f"  - {TRAIN_LIST}: {len(train_xmls)}")
print(f"  - {VAL_LIST}:   {len(val_xmls)}")

# quick sanity check
def show_sample(fp, k=5):
    print(f"\nSample from {fp}:")
    with open(fp, encoding="utf-8") as fh:
        for i, line in enumerate(fh):
            if i >= k: break
            print("  ", line.strip())
show_sample(TRAIN_LIST)
show_sample(VAL_LIST)

Found 7 ALTO XML files.
✅ Wrote lists:
  - /content/drive/MyDrive/kraken_projects/0093/lists/train.txt: 6
  - /content/drive/MyDrive/kraken_projects/0093/lists/val.txt:   1

Sample from /content/drive/MyDrive/kraken_projects/0093/lists/train.txt:
   /content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0021_mirrored.xml
   /content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0023_mirrored.xml
   /content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0024_mirrored.xml
   /content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0022_mirrored.xml
   /content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0026_mirrored.xml

Sample from /content/drive/MyDrive/kraken_projects/0093/lists/val.txt:
   /content/drive/MyDrive/kraken_projects/0093/data/export_doc5946_0093_alto_202510261625 2/0025_mirrored.xml



## 7) Training Controls

Choose your attempt, learning rate, and base model source:
	•	BASE_SOURCE = choose → type the attempt number to load from this manuscript (e.g., 13).
	•	BASE_SOURCE = upload → you’ll upload a .mlmodel as base.

Inputs to fill:
	•	ATTEMPT_ID, LEARNING_RATE, BASE_SOURCE, and if needed BASE_ATTEMPT_TO_LOAD.
Outcome: The chosen values are printed for confirmation.

Tip: On CPU, try BATCH_SIZE = 32 (bump to 48/64 if it fits).


In [21]:
#@title 🧭 Training Controls (choose or upload a base; no paths)
ATTEMPT_ID = 1            #@param {type:"number"}
LEARNING_RATE = 0.0001     #@param {type:"number"}  # set to 0 to use Kraken default
BASE_SOURCE = "upload"     #@param ["choose", "upload"]
BASE_ATTEMPT_TO_LOAD = None  #@param {type:"number"}  # attempt to load if BASE_SOURCE="choose"

# Early stopping
EARLY_STOP = "early"       #@param ["early", "dumb"]
LAG_EPOCHS = 6             #@param {type:"number"}
MIN_DELTA = 0.0001         #@param {type:"number"}

# CPU perf
import os
BATCH_SIZE = min(32, max(8, (os.cpu_count() or 2) * 2))

# Normalize LR (0 => None to use Kraken default)
LEARNING_RATE = None if (LEARNING_RATE == 0) else LEARNING_RATE

print("ATTEMPT_ID:", ATTEMPT_ID)
print("LEARNING_RATE:", LEARNING_RATE)
print("BASE_SOURCE:", BASE_SOURCE)
print("BASE_ATTEMPT_TO_LOAD:", BASE_ATTEMPT_TO_LOAD)

ATTEMPT_ID: 1
LEARNING_RATE: 0.0001
BASE_SOURCE: upload
BASE_ATTEMPT_TO_LOAD: None


## 🔗 Resolve Base Model (STRICT: choose attempt or upload; no fallback)
Picks the base .mlmodel automatically without asking for paths:
	•	If choose, it loads attempt_XX.mlmodel from this manuscript’s models/rec/.
	•	If upload, it opens a file picker and saves your upload to the right place.

Outcome: Prints the exact BASE_MODEL path. If a chosen attempt isn’t found, it lists available attempts.




In [22]:
from pathlib import Path
from google.colab import files  # type: ignore
import shutil

# Clear any stale value
if 'BASE_MODEL' in globals():
    del BASE_MODEL

def list_available_attempts(rec_dir: str):
    recp = Path(rec_dir)
    found = sorted(p for p in recp.glob("attempt_*.mlmodel"))
    if not found:
        print("ℹ️ No attempts found yet in", recp)
        return
    print("Available attempts in this manuscript:")
    for p in found:
        print("  •", p.name)

def upload_model_to_drive(dest_dir: str) -> str:
    print("📤 Upload a .mlmodel file to use as base…")
    up = files.upload()
    if not up:
        raise SystemExit("❌ No file uploaded.")
    fname = next(iter(up.keys()))
    src = f"/content/{fname}"
    dest = str(Path(dest_dir) / Path(fname).name)
    shutil.move(src, dest)
    print(f"✅ Uploaded base to: {dest}")
    return dest

BASE_MODEL = None

if BASE_SOURCE == "choose":
    chosen = Path(REC_MODELS) / f"attempt_{int(BASE_ATTEMPT_TO_LOAD):02d}.mlmodel"
    if chosen.exists():
        BASE_MODEL = str(chosen.resolve())
        print("Base = chosen attempt:", BASE_MODEL)
    else:
        print(f"❌ attempt_{int(BASE_ATTEMPT_TO_LOAD):02d}.mlmodel not found in {REC_MODELS}.")
        list_available_attempts(REC_MODELS)
        raise SystemExit("Change BASE_ATTEMPT_TO_LOAD or switch to BASE_SOURCE='upload'.")

elif BASE_SOURCE == "upload":
    BASE_MODEL = upload_model_to_drive(REC_MODELS)

print("\n✅ BASE_MODEL resolved as:", BASE_MODEL)

📤 Upload a .mlmodel file to use as base…


Saving 0088-recognizer-02.mlmodel to 0088-recognizer-02.mlmodel
✅ Uploaded base to: /content/drive/MyDrive/kraken_projects/0093/models/rec/0088-recognizer-02.mlmodel

✅ BASE_MODEL resolved as: /content/drive/MyDrive/kraken_projects/0093/models/rec/0088-recognizer-02.mlmodel


## Stage ALTO dataset locally (XMLs + images; rewrite <fileName>)
Copies each XML in the lists and its linked image to local disk (/content/alto_staged/data/), then rewrites the XML’s <fileName> to the local image name. This removes Google Drive latency.

Nothing to edit.
Outcome: Creates:
	•	LOCAL_TRAIN_LIST and LOCAL_VAL_LIST with updated local XML paths
	•	A count of staged entries

On CPU, this is the biggest speed boost. On GPU, it’s optional (still helps a bit).


In [23]:
#@title 🚀 Stage ALTO dataset locally (XMLs + images; rewrite <fileName>)
# Copies XMLs referenced by TRAIN_LIST/VAL_LIST to /content/alto_staged/data,
# copies their linked images next to them, and rewrites <fileName> in the
# copied XML to the copied image basename. Produces LOCAL_TRAIN_LIST/LOCAL_VAL_LIST.

from pathlib import Path
import xml.etree.ElementTree as ET
import shutil
import os

# Inputs from earlier cells:
# - TRAIN_LIST, VAL_LIST (XML-only lists)
# Outputs from this cell:
LOCAL_ROOT = Path("/content/alto_staged")
DATA_OUT   = LOCAL_ROOT / "data"
LOCAL_TRAIN_LIST = str(LOCAL_ROOT / "train_local.txt")
LOCAL_VAL_LIST   = str(LOCAL_ROOT / "val_local.txt")

DATA_OUT.mkdir(parents=True, exist_ok=True)

def _localname(tag: str) -> str:
    """Strip XML namespace from tag like {ns}tag → tag."""
    return tag.split('}', 1)[1] if '}' in tag else tag

def _find_alto_filename(root: ET.Element) -> str | None:
    """Return text of first <fileName> element, or None."""
    for el in root.iter():
        if _localname(el.tag) == "fileName":
            if el.text and el.text.strip():
                return el.text.strip()
    return None

def _resolve_image(xml_path: Path, file_name_text: str) -> Path | None:
    """
    Resolve the image path referenced by an ALTO <fileName>.
    Try: (xml_dir / file_name), then search by basename around xml_dir.
    """
    cand = (xml_path.parent / file_name_text)
    if cand.exists():
        return cand.resolve()
    # Fallback: search by just the name (handles absolute or weird rel paths written into <fileName>)
    base = Path(file_name_text).name
    for p in xml_path.parent.rglob(base):
        if p.exists() and p.is_file():
            return p.resolve()
    return None

def _copy_with_rewrite(xml_in: Path, dst_dir: Path) -> Path | None:
    """
    Copy xml_in and its linked image into dst_dir, rewrite <fileName> to point to copied image basename.
    Return path to copied XML, or None if image can’t be resolved.
    """
    try:
        # quick header sanity
        with open(xml_in, "r", encoding="utf-8", errors="ignore") as fh:
            if "<alto" not in fh.read(4096):
                print(f"⚠️ Not ALTO (skip): {xml_in}")
                return None

        tree = ET.parse(xml_in)
        root = tree.getroot()
        fn_text = _find_alto_filename(root)
        if not fn_text:
            print(f"⚠️ No <fileName> in: {xml_in}")
            return None

        img_in = _resolve_image(xml_in, fn_text)
        if not img_in:
            print(f"⚠️ Image not found for: {xml_in}  (<fileName>={fn_text})")
            return None

        # Prepare destinations
        dst_dir.mkdir(parents=True, exist_ok=True)
        xml_out = dst_dir / xml_in.name
        img_out = dst_dir / img_in.name

        # Copy image (first) and xml
        if not img_out.exists():
            shutil.copy2(img_in, img_out)

        # Rewrite <fileName> to local basename
        for el in root.iter():
            if _localname(el.tag) == "fileName":
                el.text = img_out.name
                break

        tree.write(xml_out, encoding="utf-8", xml_declaration=True)
        return xml_out

    except Exception as e:
        print(f"⚠️ Failed on {xml_in}: {e}")
        return None

def _collect_xmls(list_file: str) -> list[Path]:
    out: list[Path] = []
    with open(list_file, encoding="utf-8") as f:
        for line in f:
            p = Path(line.strip())
            if p.suffix.lower() == ".xml" and p.is_file():
                out.append(p.resolve())
            else:
                # ignore empties/non-xmls silently
                if line.strip():
                    # print(f"ℹ️ Skipping non-XML or missing: {line.strip()}")
                    pass
    return out

def _stage_list(orig_list: str, new_list: str) -> int:
    staged = []
    xmls = _collect_xmls(orig_list)
    for i, xml_in in enumerate(xmls, 1):
        xml_out = _copy_with_rewrite(xml_in, DATA_OUT)
        if xml_out:
            staged.append(str(xml_out))
        # Small progress ping every ~200 files
        if i % 200 == 0:
            print(f"… staged {i}/{len(xmls)}")
    with open(new_list, "w", encoding="utf-8") as f:
        f.write("\n".join(staged) + "\n")
    print(f"✅ Staged {len(staged)} / {len(xmls)} → {new_list}")
    return len(staged)

# Run staging for train/val
LOCAL_ROOT.mkdir(parents=True, exist_ok=True)
n_tr = _stage_list(TRAIN_LIST, LOCAL_TRAIN_LIST)
n_va = _stage_list(VAL_LIST,   LOCAL_VAL_LIST)

print("\nLocal staged dataset:")
print("  Root:", LOCAL_ROOT)
print("  Data dir:", DATA_OUT)
print("  TRAIN (local):", LOCAL_TRAIN_LIST, " | entries:", n_tr)
print("  VAL   (local):", LOCAL_VAL_LIST,   " | entries:", n_va)
print("\n👉 In your training cell set: _USE_LOCAL = True")

✅ Staged 6 / 6 → /content/alto_staged/train_local.txt
✅ Staged 1 / 1 → /content/alto_staged/val_local.txt

Local staged dataset:
  Root: /content/alto_staged
  Data dir: /content/alto_staged/data
  TRAIN (local): /content/alto_staged/train_local.txt  | entries: 6
  VAL   (local): /content/alto_staged/val_local.txt  | entries: 1

👉 In your training cell set: _USE_LOCAL = True



## 8) Train recognition (best checkpoint → `attempt_XX.mlmodel`)

Trains with live logs (you’ll see epochs and metrics as they happen).
The best checkpoint is saved as:
MyDrive/kraken_projects/<PROJECT_ID>/models/rec/attempt_XX.mlmodel

What to check before running:
	•	BASE_MODEL is printed correctly (from “Resolve Base Model”).
	•	_USE_LOCAL = True if you ran “Stage ALTO” (recommended on CPU).

Outcome: Live training logs; at the end you’ll see “Saved best checkpoint as …”.

Slow on CPU? Use staging, increase BATCH_SIZE, validate less often (set -F 2), or switch Colab runtime to GPU (T4).

In [27]:
import os
import subprocess
import shlex
import textwrap

# ---------------------------------------------------------------------------
# Try to infer PROJECT_ID and BASE_DIR if they are not already defined
# ---------------------------------------------------------------------------

if 'PROJECT_ID' not in globals():
    if 'BASE_MODEL' in globals() and BASE_MODEL:
        # Try to extract project ID from a path like:
        # /content/drive/MyDrive/kraken_projects/0093/models/rec/0088-recognizer-02.mlmodel
        parts = BASE_MODEL.split('/')
        if "kraken_projects" in parts:
            idx = parts.index("kraken_projects")
            if idx + 1 < len(parts):
                PROJECT_ID = parts[idx + 1]
            else:
                PROJECT_ID = "default_project"
        else:
            PROJECT_ID = "default_project"
    else:
        # Fallback – adjust if your project ID is different
        PROJECT_ID = "0093"

if 'BASE_DIR' not in globals():
    BASE_DIR = f"/content/drive/MyDrive/kraken_projects/{PROJECT_ID}"

# ---------------------------------------------------------------------------
# Train/val lists – use existing vars if defined, otherwise defaults
# ---------------------------------------------------------------------------

_TR = TRAIN_LIST if 'TRAIN_LIST' in globals() else "/content/alto_staged/train_local.txt"
_VA = VAL_LIST   if 'VAL_LIST'   in globals() else "/content/alto_staged/val_local.txt"

# ---------------------------------------------------------------------------
# Output directory for this recognizer attempt
# ---------------------------------------------------------------------------

REC_MODELS_DIR = os.path.join(BASE_DIR, "models", "rec")
os.makedirs(REC_MODELS_DIR, exist_ok=True)

# ATTEMPT_ID normalization:
#  - if missing  → "attempt_01"
#  - if int      → "attempt_XX" (zero-padded)
#  - if string   → use as-is
if 'ATTEMPT_ID' not in globals():
    ATTEMPT_ID = "attempt_01"
else:
    if isinstance(ATTEMPT_ID, int):
        ATTEMPT_ID = f"attempt_{ATTEMPT_ID:02d}"
    else:
        ATTEMPT_ID = str(ATTEMPT_ID)

out_prefix = os.path.join(REC_MODELS_DIR, ATTEMPT_ID)

print("📁 Project ID:     ", PROJECT_ID)
print("📂 BASE_DIR:       ", BASE_DIR)
print("📄 Train list:     ", _TR)
print("📄 Val list:       ", _VA)
print("📦 Output prefix:  ", out_prefix)
print("🧠 Base model:     ", BASE_MODEL if 'BASE_MODEL' in globals() else None)
print()

# ---------------------------------------------------------------------------
# Build ketos train command
# ---------------------------------------------------------------------------

# Reasonable defaults if not defined earlier
if 'LAG_EPOCHS' not in globals():
    LAG_EPOCHS = 6
if 'MIN_DELTA' not in globals():
    MIN_DELTA = 0.0001
if 'BATCH_SIZE' not in globals():
    BATCH_SIZE = 8
# LEARNING_RATE can stay None to use kraken's default

cmd = [
    "ketos", "train",
    "-o", out_prefix,
    "-f", "alto",
    "-t", _TR,
    "-e", _VA,
    "-q", "early",
    "--lag", str(int(LAG_EPOCHS)),
    "--min-delta", str(float(MIN_DELTA)),
    "-B", str(int(BATCH_SIZE)),
    "-F", "1",  # report every epoch
]

# 🔁 IMPORTANT: allow codec resize when fine-tuning an existing model
if 'BASE_MODEL' in globals() and BASE_MODEL:
    cmd += ["-i", BASE_MODEL, "--resize", "add"]

if 'LEARNING_RATE' in globals() and LEARNING_RATE is not None:
    cmd += ["-r", str(LEARNING_RATE)]

print("Running:")
print(textwrap.fill(" ".join(shlex.quote(c) for c in cmd), width=120))
print("-" * 80)

# ---------------------------------------------------------------------------
# Run training (line-by-line streaming)
# ---------------------------------------------------------------------------

proc = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,  # line-buffered
)

# Stream every line from ketos directly into the notebook output
for line in proc.stdout:
    print(line, end="")

proc.stdout.close()
proc.wait()

if proc.returncode != 0:
    raise RuntimeError("Training did not complete cleanly; scroll up to see logs.")
else:
    print("✅ Training finished successfully.")
    print("✅ Model(s) written with prefix:", out_prefix)


📁 Project ID:      0093
📂 BASE_DIR:        /content/drive/MyDrive/kraken_projects/0093
📄 Train list:      /content/drive/MyDrive/kraken_projects/0093/lists/train.txt
📄 Val list:        /content/drive/MyDrive/kraken_projects/0093/lists/val.txt
📦 Output prefix:   /content/drive/MyDrive/kraken_projects/0093/models/rec/attempt_01
🧠 Base model:      /content/drive/MyDrive/kraken_projects/0093/models/rec/0088-recognizer-02.mlmodel

Running:
ketos train -o /content/drive/MyDrive/kraken_projects/0093/models/rec/attempt_01 -f alto -t
/content/drive/MyDrive/kraken_projects/0093/lists/train.txt -e /content/drive/MyDrive/kraken_projects/0093/lists/val.txt
-q early --lag 6 --min-delta 0.0001 -B 8 -F 1 -i
/content/drive/MyDrive/kraken_projects/0093/models/rec/0088-recognizer-02.mlmodel --resize add -r 0.0001
--------------------------------------------------------------------------------
2025-11-19 22:43:36.670544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuF

KeyboardInterrupt: 

## 9) Evaluate (CER/WER)

In [ ]:

import shlex, subprocess

cmd = ["ketos", "test", "-f", "alto", "-m", final_best, VAL_LIST]
print("Running:", " ".join(shlex.quote(x) for x in cmd))
res = subprocess.run(cmd, text=True)

if res.returncode == 0:
    print("✅ Evaluation completed. See metrics above.")
else:
    raise SystemExit("❌ Evaluation failed. Check the logs above.")



### Notes
- Change only `PROJECT_ID` in Project Settings; keep **everything else** under the project folder.
- For each new attempt, adjust **ATTEMPT_ID**, **LEARNING_RATE**, and **BASE_SOURCE** in the **Training Controls** cell.
- The best checkpoint for each attempt is copied to `attempt_XX.mlmodel` so you can chain attempts cleanly.
- To warm-start a new manuscript from another one, keep **BASE_SOURCE = "auto_other"** for attempt 01 (or point to a model with `"manual"`).
- For much faster training, switch to a **T4 GPU** in Colab and set `DEVICE = "cuda"`.
